# This notebook includes scratch time series classification using RNNs

## 1) Install required packages

In [1]:
import numpy as np
import pickle as pk
from google.colab import drive
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras import models
from tensorflow.keras import Model as Model_

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## 2) Load dataset
- The dataset includes time-series data generated by a motion sensor and are collected by 10 actors with labels whether they are crouching, running, walking or something. Among several labels, here use only crouching, running and swing data. Here we load the dataset depending on window size and slide size. 

In [3]:
def motionLoad(classes, directory, window = 60, slide=1):

  file_list = os.listdir(directory)
  file_list = [i for i in file_list if any(s in i for s in classes)]
  instances, labels = [], []

  for f in file_list:
    label = [0.,0.,0.]
    index = 0
    for c, cla in enumerate(classes):
      if cla in f:
        label[c] = 1.0

    with open(directory+f) as doc:
      lines = doc.readlines()
    end = 0

    while index < len(lines):
      count = 0 
      subin = []
      if index + window < len(lines):
        if index != 0 :
          index = index - window + slide
        while count < window:
          count +=1 
          out = lines[index].split("|m ")[1].split(" |class")[0].split(" ") 
          out = [float(i) for i in out]
          subin.append(out)
          index +=1 
        instances.append(np.array(subin))
        labels.append   (np.array(label))

      else:
        index = len(lines)
  return instances, labels 

In [4]:
xv, yv = motionLoad(["Crouching", "Running", "Swing"], "/content/gdrive/My Drive/DLLab/Exercise/Motion/VAL/", window=60, slide=10)
xt, yt = motionLoad(["Crouching", "Running", "Swing"], "/content/gdrive/My Drive/DLLab/Exercise/Motion/TRAIN/", window=60, slide=10)

In [5]:
xv, yv, xt, yt = np.array(xv), np.array(yv), np.array(xt), np.array(yt)

In [6]:
xv.shape #(BATCH, TIME, FEATURES)
yv.shape #(BATCH, ONE-HOT LABELS)

(2105, 3)

## 3) Data Normalization
- For normalization, we use Min-Max normalization for each dataset. 



In [7]:
train_max = xt.max(axis=1).max(axis=0)
train_min = xt.min(axis=1).min(axis=0)
val_max   = xv.max(axis=1).max(axis=0)
val_min   = xv.min(axis=1).min(axis=0)

In [8]:
xv = (xv-val_min)/(val_max-val_min)
xt = (xt-train_min)/(train_max-train_min)

## 4) Bi-LSTMs 

In [9]:
n_timesteps,n_features = xt.shape[1], xt.shape[2]
n_outputs = yt.shape[1]

In [10]:
class LSTMs(Model_):
  def __init__(self):
    super(LSTMs, self).__init__()

    self.Layer1=Bidirectional(LSTM(units=128,input_shape=[n_timesteps, n_features]))
    self.Layer2=Dense(units=128, activation='relu')
    self.Layer3=Dense(n_outputs, activation='softmax')

  def call(self, data):
    output=self.Layer1(data)
    output=self.Layer2(output)
    output=self.Layer3(output) 
    return output

In [15]:
class Optimizer:
  def __init__(self, model, batch_size = 10, loss = tf.keras.losses.CategoricalCrossentropy, lr = 0.001, opt=tf.keras.optimizers.Adam):
    self.model     = model
    self.loss      = loss()
    self.optimizer = opt(learning_rate = lr)
    self.batch_size = batch_size
    self.train_loss     = tf.keras.metrics.Mean(name='train_loss')
    self.train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    self.test_loss     = tf.keras.metrics.Mean(name='test_loss')
    self.test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
    
  @tf.function
  def train_step(self, x , y):
    with tf.GradientTape() as tape:
      predictions = self.model(x)
      loss = self.loss(predictions, y)
    gradients = tape.gradient(loss, self.model.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
    self.train_loss(loss)
    self.train_accuracy(y, predictions)
    return loss

  @tf.function
  def test_step(self, x , y):
    predictions = self.model(x)
    loss = self.loss(predictions, y)
    self.test_loss(loss)
    self.test_accuracy(y, predictions) 
    
  def train (self):
    for mbX, mbY in self.train_ds:
      self.train_step(mbX, mbY)

  def test  (self):
    for mbX, mbY in self.test_ds:
      self.test_step(mbX, mbY)  

  def run   (self, dataX, dataY, testX, testY, epochs, verbose=2): #verbose=2
    historyTR_LOSS = []
    historyTS_LOSS = []
    historyTR_ACC = []
    historyTS_ACC = []

    template = '{} {}, {}: {}, {}: {}, {}: {}, {}: {}'
    self.train_ds = tf.data.Dataset.from_tensor_slices((dataX, dataY)).shuffle(16000).batch(self.batch_size)
    self.test_ds  = tf.data.Dataset.from_tensor_slices((testX,testY)).batch(self.batch_size)

    for i in range(epochs):
      self.train ()
      self.test  ()
      if verbose > 0: 
        print(template.format("epoch: ", i+1,
                              "TRAIN LOSS: ", self.train_loss.result(),
                              "TEST LOSS: " , self.test_loss.result(),
                              "TRAIN ACC: " , self.train_accuracy.result().numpy()*100,
                              "TEST ACC: "  , self.test_accuracy.result().numpy()*100))
      temp = '{}'
    
      historyTR_LOSS.append(float(temp.format(self.train_loss.result().numpy())))
      historyTS_LOSS.append(float(temp.format(self.train_loss.result().numpy())))
      historyTR_ACC.append(float(temp.format(self.train_accuracy.result().numpy()*100)))
      historyTS_ACC.append(float(temp.format(self.test_accuracy.result().numpy()*100)))

      self.train_loss.reset_states()
      self.test_loss.reset_states()
      self.train_accuracy.reset_states()
      self.test_accuracy.reset_states()
    return historyTR_LOSS, historyTS_LOSS, historyTR_ACC, historyTS_ACC

In [19]:
model = LSTMs()
opt    = Optimizer (model, batch_size = 256, lr = 0.001, loss = tf.keras.losses.CategoricalCrossentropy, opt=tf.keras.optimizers.Adam)
tl, vl, ta, va = opt.run (xt, yt, xv, yv, 100, verbose=1)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch:  1, TRAIN LOSS: : 10.064024925231934, TEST LOSS: : 7.368531227111816, TRAIN ACC: : 47.0449298620224, TEST ACC: : 62.61282563209534
epoch:  2, TRAIN LOSS: : 6.886173248291016, TEST LOSS: : 6.761136531829834, TRAIN ACC: : 58.03861618041992, TEST ACC: : 63.46793174743652
epoch:  3, TRAIN LOSS: : 6.327403545379639, TEST LOSS: : 6.570125102996826, TRAIN ACC: : 60.97891926765442, TEST ACC: : 64.79809880256653
epoch:  4, TRAIN LOSS: : 6.250520706176758, TEST LOSS: : 6.588329792022705, TRAIN ACC: : 61.34